# Introduction

This notebook explores the potential to calculate wind speed and direction on the ground from a vehicle in flight.

The **primary application** focus is to gather magnitude and direction of the wind on the ground from a fixed wing aircraft preparing to land.

Generally it is wise to take off and land both facing into the wind. This minimizes the amount of kinetic energy one carries relative to the ground. 

$$Kinetic Energy =\frac{1}{2}mass * velocity^{2}$$

The less energy, the safer aircraft operations close to the ground generally are. In addition to reduced kinetic energy, it also allows for shorter takeoff and landing distances. Which can be incredibly useful when operating out of unimproved runways or in the backcountry.

# Theory

Determining the wind direction and speed on the ground while in the air amounts to separating:
1. Air moving relative to the aircraft
2. Air moving relative to the ground

This is tricky because the only measurable air while in flight, is the air relative to the aircraft. There are other quantites to measure though, like position, that may be useful to compare to measurements of the air to try and understand the wind velocity on the ground.

### Initial Idea

My first thought is to measure the dynamic pressure via static pressure and total pressure measurements at the aircraft then subtract it from the ground speed measured using GPS. Direction can be gathered from the GPS data. This results in a vector that gives the difference between the ground and air velocities in every direction measured. If this is done in a full circle, it may provide enough information to calculate the wind speed and direction on the ground.

In [8]:
import numpy as np
import pandas as pd

In [1]:
ias = [100,110,100,90]

### Airspeed

Airspeed measured at the aircraft is called IAS or indicated airspeed. This speed represents the dynamic pressure, Or the difference between total pressure and static pressure. Read about it here: https://en.wikipedia.org/wiki/Indicated_airspeed

Unfortunately indicated airspeed is not the same as true airspeed. So I will not be able to get a true airspeed reading using only dynamic pressure.
